# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import tifffile as tiff
from skimage.transform import resize
from pathlib import Path
import kagglehub
import warnings
warnings.filterwarnings('ignore')

# Configuration

In [3]:
IMG_SIZE = (64, 64)
NUM_CLASSES = 10

CLASSES = [
    'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway',
    'Industrial', 'Pasture', 'PermanentCrop', 'Residential',
    'River', 'SeaLake'
]

SPECTRAL_BANDS = [
    'Coastal Aerosol', 'Blue', 'Green', 'Red', 'Red Edge 1',
    'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4',
    'Water Vapor', 'SWIR Cirrus', 'SWIR 1', 'SWIR 2'
]


print("Config Setup completed...")

Config Setup completed...


# Download data from kaggle

In [4]:
dataset_path = kagglehub.dataset_download("apollo2506/eurosat-dataset")

print("Dataset downloaded to:", dataset_path)

Using Colab cache for faster access to the 'eurosat-dataset' dataset.
Dataset downloaded to: /kaggle/input/eurosat-dataset


# Data Loading....


In [5]:
def load_satellite_data(dataset_dir, img_size=IMG_SIZE):

    print("Loading Data...")

    data = []
    labels = []
    class_folders = sorted(os.listdir(dataset_dir))

    for class_name in class_folders:
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue


        for file_name in os.listdir(class_path):
            img_path = os.path.join(class_path, file_name)
            try:
                # Read 13-band images
                img = tiff.imread(img_path)
                # Ensure image contains 13 band
                if img.shape[-1] != 13:
                    print(f"⚠️ Skipping {img_path}: unexpected band count {img.shape[-1]}")
                    continue

                img = resize(img, (img_size[0], img_size[1], 13),
                             preserve_range=True,
                             anti_aliasing=True)

                # Normalize values 0 => 1
                img = img.astype(np.float32)
                img /= img.max()

                data.append(img)
                labels.append(class_name)

            except Exception as e:
                print(f"⚠️ Skipping {img_path}: {e}")

    X = np.array(data)
    Y = np.array(labels)
    return X, Y

# Load data
# DATASET_PATH ="/root/.cache/kagglehub/datasets/apollo2506/eurosat-dataset/versions/6/EuroSATallBands"
DATASET_PATH ="/kaggle/input/eurosat-dataset/EuroSATallBands"

X, Y = load_satellite_data(DATASET_PATH)

print(f"\n{len(X)} images loaded successfully.")
print(f"Data shape: {X.shape}")
print(f"Classes found: {sorted(CLASSES)}")


Loading Data...

27597 images loaded successfully.
Data shape: (27597, 64, 64, 13)
Classes found: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


# Data Exploration

In [6]:
print("## BASIC DATA EXPLORATION ##")

unique_classes, counts = np.unique(Y, return_counts=True)
# print(unique_classes)
# print(counts)
class_distribution = dict(zip(unique_classes, counts))

print("\nClass Distribution:")
for class_name, count in class_distribution.items():
    print(f"{class_name}: {count} samples")



## BASIC DATA EXPLORATION ##

Class Distribution:
AnnualCrop: 3000 samples
Forest: 3000 samples
HerbaceousVegetation: 3000 samples
Highway: 2500 samples
Industrial: 2500 samples
Pasture: 2000 samples
PermanentCrop: 2500 samples
Residential: 3000 samples
River: 2500 samples
SeaLake: 3597 samples


In [10]:
# Assuming X is the loaded data with shape (N, 64, 64, 13)

# Task 3: Calculate Global Mean and Standard Deviation
print("\nCalculating global mean (μ) and standard deviation (σ) for all 13 bands...")

# Calculate Mean (average over batch, width, and height dimensions)
# Axis 0 is the batch (image count), Axis 1 is height, Axis 2 is width.
GLOBAL_MEAN = X.mean(axis=(0, 1, 2))

# Calculate Standard Deviation (std over batch, width, and height dimensions)
GLOBAL_STD = X.std(axis=(0, 1, 2))

# Display Results
# Note: SPECTRAL_BANDS was defined earlier and contains 13 names.
stats_df = pd.DataFrame({
    'Band Name': SPECTRAL_BANDS,
    'Global Mean (μ) [0-1 scaled]': GLOBAL_MEAN,
    'Global Std Dev (σ) [0-1 scaled]': GLOBAL_STD
})

print("\n--- Global Normalization Parameters (Deliverable) ---")
print("These statistics are calculated on the 0-1 Min-Max scaled data loaded by the previous step.")
print(stats_df)



Calculating global mean (μ) and standard deviation (σ) for all 13 bands...

--- Global Normalization Parameters (Deliverable) ---
These statistics are calculated on the 0-1 Min-Max scaled data loaded by the previous step.
          Band Name  Global Mean (μ) [0-1 scaled]  \
0   Coastal Aerosol                      0.148422   
1              Blue                      0.148422   
2             Green                      0.136953   
3               Red                      0.109659   
4        Red Edge 1                      0.119203   
5        Red Edge 2                      0.148422   
6        Red Edge 3                      0.148422   
7               NIR                      0.148422   
8        Red Edge 4                      0.074297   
9       Water Vapor                      0.002212   
10      SWIR Cirrus                      0.148422   
11           SWIR 1                      0.120967   
12           SWIR 2                      0.148422   

    Global Std Dev (σ) [0-1 scaled

In [11]:
# Save the critical normalization parameters
# We save the calculated values for use in the next milestone's normalization step.
stats_df[['Band Name', 'Global Mean (μ) [0-1 scaled]', 'Global Std Dev (σ) [0-1 scaled]']].to_csv('normalization_parameters.csv', index=False)
print("\n✅ Normalization parameters saved to 'normalization_parameters.csv'")


# Summary Report Generation
total_images = len(X)
# Reusing the 'counts' variable from the previous cell for class distribution statistics
mean_count = counts.mean()
std_count = counts.std()

# Simple heuristic check for reporting: high std deviation relative to mean indicates noticeable imbalance
is_imbalanced = (std_count / mean_count) > 0.1

cleaning_report = {
    "Total Images Loaded (After Filtering)": total_images,
    "Class Distribution Imbalance Noted (Std/Mean > 0.1)": is_imbalanced,
    "Global Mean Calculated (Bands 1-5)": list(np.round(GLOBAL_MEAN[:5], 4)),
    "Global Std Dev Calculated (Bands 1-5)": list(np.round(GLOBAL_STD[:5], 4)),
    "Readiness for Preprocessing": "True (Normalization parameters are ready for Milestone 3)"
}

print("\n--- Milestone 2 Final Summary Report ---")
for k, v in cleaning_report.items():
    print(f"{k:<55}: {v}")



✅ Normalization parameters saved to 'normalization_parameters.csv'

--- Milestone 2 Final Summary Report ---
Total Images Loaded (After Filtering)                  : 27597
Class Distribution Imbalance Noted (Std/Mean > 0.1)    : True
Global Mean Calculated (Bands 1-5)                     : [np.float32(0.1484), np.float32(0.1484), np.float32(0.137), np.float32(0.1097), np.float32(0.1192)]
Global Std Dev Calculated (Bands 1-5)                  : [np.float32(0.2772), np.float32(0.2241), np.float32(0.1778), np.float32(0.1601), np.float32(0.1755)]
Readiness for Preprocessing                            : True (Normalization parameters are ready for Milestone 3)
